<a href="https://colab.research.google.com/github/johanhoffman/DD2363_VT24/blob/ivan-private-Lab1/Lab1/ivan-private_lab1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Lab 1: Matrix factorization**
**Ivan Zivkovic**

# **Abstract**



```
# This is formatted as code
```

Short summary of the lab report. State the objectives, methods used, main results and conlusions.

#**About the code**

A short statement on who is the author of the file, and if the code is distributed under a certain license.

In [1]:
"""This program is a template for lab reports in the course"""
"""DD2363 Methods in Scientific Computing, """
"""KTH Royal Institute of Technology, Stockholm, Sweden."""

# Copyright (C) 2020 Johan Hoffman (jhoffman@kth.se)

# This file is part of the course DD2365 Advanced Computation in Fluid Mechanics
# KTH Royal Institute of Technology, Stockholm, Sweden
#
# This is free software: you can redistribute it and/or modify
# it under the terms of the GNU Lesser General Public License as published by
# the Free Software Foundation, either version 3 of the License, or
# (at your option) any later version.

# This template is maintained by Johan Hoffman
# Please report problems to jhoffman@kth.se

'KTH Royal Institute of Technology, Stockholm, Sweden.'

# **Set up environment**

To have access to the neccessary modules you have to run this cell. If you need additional modules, this is where you add them.

In [2]:
# Load neccessary modules.
from google.colab import files

import time
import numpy as np

#try:
#    from dolfin import *; from mshr import *
#except ImportError as e:
#    !apt-get install -y -qq software-properties-common
#    !add-apt-repository -y ppa:fenics-packages/fenics
#    !apt-get update -qq
#    !apt install -y --no-install-recommends fenics
#    from dolfin import *; from mshr import *

#import dolfin.common.plotting as fenicsplot

from matplotlib import pyplot as plt
from matplotlib import tri
from matplotlib import axes
from mpl_toolkits.mplot3d import Axes3D

# **Introduction**

Give a short description of the problem investigated in the report, and provide some background information so that the reader can understand the context. If appropriate, include a discussion on the state of the art.

Briefly describe what method you have chosen to solve the problem, and justify why you selected that method.

Here you can express mathematics through Latex syntax, and use hyperlinks for references.

[Hyperlink to DD2363 course website.](https://kth.instructure.com/courses/17068)

$
{\displaystyle \frac{\partial u}{\partial t}} + u\cdot \nabla u +\nabla p = f, \quad \nabla \cdot u=0$



# **Method**

Describe the methods you used to solve the problem. This may be a combination of text, mathematical formulas (Latex), algorithms (code), data and output.  

## **Sparse matrix-vector product**

In [3]:
class SparseMatrix:
    '''
        :param A: dense matrix of shape (m, n)
        :return SparseMatrix
    '''
    def __init__(self, A: np.ndarray):
        m, n = A.shape
        val = []
        col_idx = []
        row_ptr = []
        for i in range(m):
            for j in range(n):
                if A[i, j] != 0:
                    if len(row_ptr) == i:
                        row_ptr.append(len(val))
                    val.append(A[i, j])
                    col_idx.append(j)

        row_ptr.append(len(val))
        self.val = np.array(val)
        self.col_idx = np.array(col_idx)
        self.row_ptr = np.array(row_ptr)


def sparse_matrix_vector_product(A: SparseMatrix, x: np.ndarray):
    n = x.shape[0]
    b = np.zeros((n,))

    for i in range(n):
        for j in range(A.row_ptr[i], A.row_ptr[i+1]):
            b[i] = b[i] + A.val[j] * x[A.col_idx[j]]

    return b

## **QR factorization**

In [4]:
def modified_gram_schmidt_QR_factorization(A: np.ndarray) -> (np.ndarray, np.ndarray):
    _, n = A.shape
    Q = np.zeros((n, n))
    R = np.zeros((n, n))


    for j in range(n):
        v = A[:, j]

        for i in range(j):
            R[i, j] = Q[:, i].T @ v
            v = v - R[i, j] * Q[:, i]

        R[j, j] = np.sqrt(v.T @ v) #norm of v

        Q[:, j] = v / R[j, j]

    return Q, R

## **Direct solver Ax=b**

In [5]:
def backwards_substitution(R: np.ndarray, b: np.ndarray) -> np.ndarray:
    # backwards substitution algorithm
    _, n = R.shape
    x = np.zeros(n)
    x[n-1] = b[n-1] / R[(n-1, n-1)]
    for i in range(n-2, -1, -1):
        x[i] = b[i]
        for j in range(i+1, n):
            x[i] = x[i] - R[i, j] * x[j]
        x[i] = x[i] / R[i, i]

    return x

def direct_solver(A: np.ndarray, b: np.ndarray) -> np.ndarray:

    Q, R = modified_gram_schmidt_QR_factorization(A)

    # Q.T = Q^-1 since Q is orthogonal
    return backwards_substitution(R, Q.T @ b)

## **QR eigenvalue algorithm**

In [6]:
def rayleigh_quotient(A: np.ndarray, x: np.ndarray) -> float:
    return (x.T @ (A @ x)) / (x.T @ x)


def qr_algorithm(A: np.ndarray, k: int) -> Tuple[np.ndarray, np.ndarray]:
    n = A.shape[0]
    U = np.identity(n)


    for i in range(k):
        Q, R = modified_gram_schmidt_QR_factorization(A)
        A = R @ Q
        U = U @ Q

    return A, U




```
# Detta formeras som kod
```

# **Results**

Present the results. If the result is an algorithm that you have described under the *Methods* section, you can present the data from verification and performance tests in this section. If the result is the output from a computational experiment this is where you present a selection of that data.

## **Sparse matrix-vector product**

In [7]:
n = 6
A = np.random.randint(0, 10, size=(n,n))
x = np.random.randint(0, 10, size=(n,))
sparse_A = SparseMatrix(A)

# Test 1
Ax = A @ x
sparse_Ax = sparse_matrix_vector_product(sparse_A, x)
diff = np.linalg.norm(Ax - sparse_Ax)

print(f"A:\n{A}")
print(f"x: {x}")
print(f"Ax: {Ax}")
print(f"sparse_Ax: {sparse_Ax}")
print(f"norm of difference: {diff}")

A:
[[7 4 6 7 0 3]
 [0 9 2 8 5 3]
 [8 9 1 5 7 0]
 [8 3 1 5 6 9]
 [0 9 1 5 6 4]
 [4 3 6 6 5 6]]
x: [9 0 7 2 1 5]
Ax: [134  50  96 140  43 125]
sparse_Ax: [134.  50.  96. 140.  43. 125.]
norm of difference: 0.0


## **QR factorization**

In [8]:
def TEST_modified_gram_schmidt_QR_factorization():
    A = np.array( [[2, -1],
               [-1, 2]] )

    _, n = A.shape
    assert A.shape == (n, n)

    Q, R = modified_gram_schmidt_QR_factorization(A)

    # test 1
    assert np.allclose(R, np.triu(R))

    # test 2
    assert np.allclose(Q.T @ Q, np.identity(n))

    # test 3
    assert np.allclose(A, Q @ R)



TEST_modified_gram_schmidt_QR_factorization() #throws assertion error if any testcase did not pass
print("All tests passed")

All tests passed


## **Direct solver Ax=b**

In [9]:
def TEST_direct_solver():
    A = np.array( [[2, -1],
               [-1, 2]] )

    b = np.array( [1, 0] )

    TEST_modified_gram_schmidt_QR_factorization()

    _, n = A.shape
    assert A.shape == (n, n)
    assert b.shape == (n, )

    x = direct_solver(A, b)
    assert x.shape == (n, )

    # test 1
    assert np.allclose(A @ x, b)

    # test 2
    y = np.array( [5, 7] )
    b = A @ y
    x = direct_solver(A, b)
    assert np.allclose(x, y)


TEST_direct_solver() #throws assertion error if any testcase did not pass
print("All tests passed")

All tests passed


## **QR eigenvalue algorithm**

In [10]:
# Test 1
n = 3
A = np.random.randint(0, 10, size=(n, n)) #TODO, check if qr_algorithm works for any square matrix

A2, U = qr_algorithm(A, 1000)

print("Test 1")
for i in range(n):
    lambda_i = A2[i, i]
    det = np.linalg.det(A - (lambda_i * np.identity(n)))
    print(f"det(A - lambda_{i+1} * I) = {det}")

print("\nTest 2")
for i in range(n):
    lambda_i = A2[i, i]
    v_i = U[:, i]
    norm = np.linalg.norm(A @ v_i - lambda_i * v_i)
    print(f"||A * v_{i+1} - lambda_{i+1} * v_{i+1}|| = {norm}")


Test 1
det(A - lambda_1 * I) = -3.6914743172244234e-13
det(A - lambda_2 * I) = -4.0865660294043553e-14
det(A - lambda_3 * I) = 1.4032109348778375e-30

Test 2
||A * v_1 - lambda_1 * v_1|| = 9.607118501968132e-15
||A * v_2 - lambda_2 * v_2|| = 3.000000000000002
||A * v_3 - lambda_3 * v_3|| = 12.27057756846608


# **Discussion**

Summarize your results and your conclusions. Were the results expected or surprising. Do your results have implications outside the particular problem investigated in this report?